In [1]:
from google.colab import drive
drive.mount('/content/drive',force_remount=True)

Mounted at /content/drive


In [2]:
import numpy as np
import pandas as pd
import glob
import pickle

## 데이터프레임 형태 정리

In [3]:
# 경로 설정
path = "/content/drive/MyDrive/DSL/[프로젝트] 모델링/A3NCF/data preprocessing/rawdata_for_A3NCF/"

In [4]:
# reviews_five 불러오기 (review 5개 이상 & rating 5개 이상)
df = pd.read_csv(path + "reviews_five.csv")
df.head()

,name,address,user_id,rating,review,cafe
0,고로커피로스터스,봉천동,100i22n,4.0,커피랑 원두 맛남,고로커피로스터스 봉천동
1,나따오비까 대치점,대치동,100i22n,3.0,에그타르트 클래식 ☆☆☆\n에그타르트 크림치즈 ☆☆☆.5\n\n시나몬향이 나고 빵 ...,나따오비까 대치점 대치동
2,뉴욕택시디저트,봉천동,100i22n,2.0,나한테는 디저트가 너무 달았다 ㅜ,뉴욕택시디저트 봉천동
3,델트,구의동,100i22n,3.0,플랫화이트 ☆☆☆\n\n그럭저럭,델트 구의동
4,카페산다,봉천동,100i22n,3.0,NaN,카페산다 봉천동


In [5]:
# 전체 리뷰 개수 (rows 7041, user 804명, cafe 2847개)
rows = df.shape[0]
user = df["user_id"].unique().size
cafe = df["cafe"].unique().size
print(f"rows: {rows}, user: {user}, cafe: {cafe}")

rows: 7041, user: 804, cafe: 2847


- 사용할 데이터

|df|df_input|
| --- | --- |
| name | x |
| address | x |
| user_id | userindex |
| rating | rating |
| review | reviews |
| cafe | itemindex |

In [6]:
# name, address 제거
df.drop(columns=["name", "address"], inplace=True)
df.head()

,user_id,rating,review,cafe
0,100i22n,4.0,커피랑 원두 맛남,고로커피로스터스 봉천동
1,100i22n,3.0,에그타르트 클래식 ☆☆☆\n에그타르트 크림치즈 ☆☆☆.5\n\n시나몬향이 나고 빵 ...,나따오비까 대치점 대치동
2,100i22n,2.0,나한테는 디저트가 너무 달았다 ㅜ,뉴욕택시디저트 봉천동
3,100i22n,3.0,플랫화이트 ☆☆☆\n\n그럭저럭,델트 구의동
4,100i22n,3.0,NaN,카페산다 봉천동


In [7]:
# 열 이름 및 순서 변경
df.columns = ['userIndex', 'rating', 'reviews', 'itemIndex']
df = df[['userIndex', 'itemIndex', 'rating', 'reviews']]
df.head()

,userIndex,itemIndex,rating,reviews
0,100i22n,고로커피로스터스 봉천동,4.0,커피랑 원두 맛남
1,100i22n,나따오비까 대치점 대치동,3.0,에그타르트 클래식 ☆☆☆\n에그타르트 크림치즈 ☆☆☆.5\n\n시나몬향이 나고 빵 ...
2,100i22n,뉴욕택시디저트 봉천동,2.0,나한테는 디저트가 너무 달았다 ㅜ
3,100i22n,델트 구의동,3.0,플랫화이트 ☆☆☆\n\n그럭저럭
4,100i22n,카페산다 봉천동,3.0,NaN


## reviews column 전처리
- 문장부호
- stopword 
- 전체 데이터셋에서 10회 이하 등장한 단어들  
=> 제외

### 사전 준비
- review 컬럼 분리
- NLP 관련 library 호출

In [8]:
# df_review: df에서 reviews column만 분리
df_review = df.reviews
df_review[:5]

0                                            커피랑 원두 맛남
1    에그타르트 클래식 ☆☆☆\n에그타르트 크림치즈 ☆☆☆.5\n\n시나몬향이 나고 빵 ...
2                                   나한테는 디저트가 너무 달았다 ㅜ
3                                    플랫화이트 ☆☆☆\n\n그럭저럭
4                                                  NaN
Name: reviews, dtype: object

In [9]:
# list_review: df_review의 자료구조 변경 (series -> list)
list_review = df_review.to_list()
list_review[:5]

['커피랑 원두 맛남',
 '에그타르트 클래식 ☆☆☆\n에그타르트 크림치즈 ☆☆☆.5\n\n시나몬향이 나고 빵 부분은 바삭바삭하나 계란 맛이 좀 적게 느껴진다',
 '나한테는 디저트가 너무 달았다 ㅜ',
 '플랫화이트 ☆☆☆\n\n그럭저럭',
 nan]

In [10]:
# list_review 내 요소의 자료형을 전부 string으로 바꾸기
for i, review in enumerate(list_review):
    review = str(review)
    list_review[i] = review

list_review[:5]

['커피랑 원두 맛남',
 '에그타르트 클래식 ☆☆☆\n에그타르트 크림치즈 ☆☆☆.5\n\n시나몬향이 나고 빵 부분은 바삭바삭하나 계란 맛이 좀 적게 느껴진다',
 '나한테는 디저트가 너무 달았다 ㅜ',
 '플랫화이트 ☆☆☆\n\n그럭저럭',
 'nan']

In [11]:
#konlpy 설치 (mecab 제외). 3~40초 정도 소요
%%bash
apt-get update
apt-get install g++ openjdk-8-jdk python-dev python3-dev
pip3 install JPype1
pip3 install konlpy

Get:1 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease [3,626 B]
Hit:2 http://archive.ubuntu.com/ubuntu bionic InRelease
Get:3 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Get:4 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease [15.9 kB]
Ign:5 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Get:6 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease [1,581 B]
Get:7 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Hit:8 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Hit:9 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Get:10 http://archive.ubuntu.com/ubuntu bionic-backports InRelease [83.3 kB]
Hit:11 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu bionic InRelease
Get:12 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Packages [913 k

In [12]:
# java 경로 설정
%env JAVA_HOME "/usr/lib/jvm/java-8-openjdk-amd64"

env: JAVA_HOME="/usr/lib/jvm/java-8-openjdk-amd64"


In [13]:
# 필요 라이브러리 불러오기
import os, json
from tqdm.notebook import tqdm
from konlpy.tag import Okt

In [14]:
okt = Okt()

### 품사를 통한 stopword 제거 
- 문장부호, 조사, 접미사, 선어말어미, 어미, 외국어 및 기타 기호, 이메일, URL
- 참고: [형태소 분석기 품사 태그 표](https://velog.io/@hsoh0423/%ED%98%95%ED%83%9C%EC%86%8C-%EB%B6%84%EC%84%9D%EA%B8%B0-%ED%92%88%EC%82%AC-%ED%83%9C%EA%B7%B8%ED%91%9C)

In [15]:
def stopwordTokenization(sentence):
    stop = ['Punctuation', 'Josa', 'Suffix', 'PreEomi', 'Eomi', 'Foreign', 'Email', 'URL']
    token = okt.pos(sentence)
    token = [i for i, j in token if j not in stop]
    return token

In [16]:
review_tokenized = []

for i in tqdm(list_review):
    review_tokenized.append(stopwordTokenization(i))

  0%|          | 0/7041 [00:00<?, ?it/s]

In [17]:
review_tokenized[:5]

[['커피', '원두', '맛남'],
 ['에그타르트',
  '클래식',
  '에그타르트',
  '크림',
  '치즈',
  '5',
  '시나몬',
  '향',
  '나고',
  '빵',
  '부분',
  '바삭바삭하나',
  '계란',
  '맛',
  '좀',
  '적게',
  '느껴진다'],
 ['나', '는', '디저트', '너무', '달았다', 'ㅜ'],
 ['플랫', '화이트', '그럭저럭'],
 ['nan']]

### 전체 데이터셋에서 3회 이하 등장한 단어들 제거

In [18]:
# bag-of-word
def build_bag_of_words(review):

    word_count = {}
    bow = []
    
    for sentence in review:     # review_tokenized에서 list 하나씩 꺼내기
        for word in sentence:   # 꺼낸 list에서 word 하나씩 꺼내기
            if word not in word_count.keys():
                word_count[word] = len(word_count)  
                # BoW에 전부 기본값 1을 넣는다.
                bow.insert(len(word_count) - 1, 1)
            else:
                # 재등장하는 단어의 인덱스
                index = word_count.get(word)
                # 재등장한 단어는 해당하는 인덱스의 위치에 1을 더한다.
                bow[index] = bow[index] + 1

    return word_count, bow

In [19]:
vocab, bow = build_bag_of_words(review_tokenized)
print('vocabulary :', vocab)
print('bag of words vector :', bow)

vocabulary : {'커피': 0, '원두': 1, '맛남': 2, '에그타르트': 3, '클래식': 4, '크림': 5, '치즈': 6, '5': 7, '시나몬': 8, '향': 9, '나고': 10, '빵': 11, '부분': 12, '바삭바삭하나': 13, '계란': 14, '맛': 15, '좀': 16, '적게': 17, '느껴진다': 18, '나': 19, '는': 20, '디저트': 21, '너무': 22, '달았다': 23, 'ㅜ': 24, '플랫': 25, '화이트': 26, '그럭저럭': 27, 'nan': 28, '안달': 29, '맛있어요': 30, '조금': 31, '늦게': 32, '가니': 33, '베이글': 34, '없넹': 35, 'ㅜㅜ': 36, '흑': 37, '좋다': 38, '분위기': 39, '많은': 40, '기대': 41, '했나': 42, '음식': 43, '왜케': 44, '맛없': 45, 'ㅠㅜ': 46, '테라스': 47, '넘': 48, '좋은데': 49, '앞': 50, '사람': 51, '담배': 52, '핀다': 53, '냠냠': 54, '좋': 55, '앙': 56, 'ㅎㅎ': 57, '뇨끼': 58, '존맛': 59, '탱': 60, '주차': 61, '1시간': 62, '지원': 63, '돼서': 64, '좋아요': 65, '인테리어': 66, '와': 67, '스콘': 68, '맛있당': 69, '따뜻한': 70, '마시': 71, '따': 72, '까눌레': 73, '완전': 74, 'ㅋㅋㅋㅋㅋ': 75, '무난': 76, '카페': 77, '음료': 78, '양': 79, '엄청': 80, '많고': 81, '사장': 82, '젠틀': 83, '멋': 84, '지심': 85, '여기': 86, '많아서': 87, '겁나': 88, '밀려서': 89, '나옴': 90, 'ㅜㅋㅋㅋㅋㅋ': 91, '그리고': 92, '여자': 93, '알바생': 94, '4': 95, '가지': 96, '없음'

In [20]:
# vocab: dict -> dataframe
df_vocab = pd.DataFrame(vocab, index=["index"])
df_vocab = df_vocab.transpose()
df_vocab["word"] = df_vocab.index
df_vocab.set_index(keys="index", inplace=True)
df_vocab

,word
index,
0,커피
1,원두
2,맛남
3,에그타르트
4,클래식
...,...
15343,와글와글
15344,의무
15345,됐나요


In [21]:
# bow: list -> series
df_bow = pd.Series(bow)
df_bow

0        1522
1         113
2          36
3          38
4          13
         ... 
15343       1
15344       1
15345       1
15346       1
15347       1
Length: 15348, dtype: int64

In [22]:
# df_vocab + df_bow
df_vocab["count"] = df_bow
df_vocab.sort_values(["count"], ascending=False, inplace=True)
df_vocab

,word,count
index,,
0,커피,1522
15,맛,1367
22,너무,1016
183,더,978
77,카페,816
...,...,...
9241,많았던거,1
9240,닫혀있었어요,1
9239,닫는거,1


In [23]:
higher = df_vocab[df_vocab["count"] >= 3]
lower = df_vocab[df_vocab["count"] < 3]
higher_sum = higher["count"].sum()
lower_sum = lower["count"].sum()
print("3번 이상 출현한 단어들의 총 개수:", higher_sum, "\n" "3번 미만 출현한 단어들의 총 개수:", lower_sum)

3번 이상 출현한 단어들의 총 개수: 84351 
3번 미만 출현한 단어들의 총 개수: 13140


In [24]:
# df_vocab: 전체 리뷰에서 3번 이상 등장한 단어들의 bow
df_vocab = df_vocab[df_vocab["count"] >= 3]
df_vocab

,word,count
index,,
0,커피,1522
15,맛,1367
22,너무,1016
183,더,978
77,카페,816
...,...,...
908,뷔릴레,3
12952,맨손,3
6848,잡담,3


In [25]:
# list_vocab: 전체 리뷰의 단어 집합
list_vocab = list(df_vocab["word"])
len(list_vocab)

4429

In [26]:
# review_tokenized에서 list_vocab에 없는 단어들(3회 미만 등장 단어들) 삭제
for review in review_tokenized:             # 전체 리뷰에서 하나의 리뷰를 꺼낸다
    for i, word in enumerate(review):       # 하나의 리뷰에서 하나의 토큰(word)을 꺼낸다
        if word not in list_vocab:          # 그 토큰이 list_vocab에 있는 단어가 아니라면
            #print(word)             
            del review[i]                   # 해당 토큰 삭제

In [27]:
review_tokenized[:5]

[['커피', '원두', '맛남'],
 ['에그타르트',
  '클래식',
  '에그타르트',
  '크림',
  '치즈',
  '5',
  '시나몬',
  '향',
  '나고',
  '빵',
  '부분',
  '계란',
  '맛',
  '좀',
  '적게',
  '느껴진다'],
 ['나', '는', '디저트', '너무', '달았다', 'ㅜ'],
 ['플랫', '화이트', '그럭저럭'],
 ['nan']]

In [28]:
# 잘 삭제되었는지 확인
for review in review_tokenized:     # 전체 리뷰에서 하나의 리뷰를 꺼낸다
    for word in review:             # 하나의 리뷰에서 하나의 토큰(word)을 꺼낸다
        if word not in list_vocab:  # 그 토큰이 list_vocab에 있는 단어가 아니라면
            print(word)             # 출력해줘 -> 아무것도 출력 안되면 OK

ㅜㅋㅋㅋ
이인
티비
엽서
나온다
알았음
r
구독
미각
싫다면
주십니다
남는샷
도구
어른
있으신
재미있을
새것의
두어
쩖
상시
벤치
바라볼
않다는
듁스
부리는
ㅠㅠㅠㅠㅠㅠㅠㅠㅠ
오드리
ㅠㅜㅜ
찾으
#문래동카페
#갬성카페
신림로
희
쏭
강남대로
유당불내증
캐슈넛
건무
다이어트
궁금궁금
신다면
사평대로
212
느꼈네요
마켓
편이였네요
만든것
단길
실화
딴
느껴졌다
독
즐거웠다
불편함이
어지간하면
하겠음
페퍼
땅땅
로빈
우지
탑승
잡아
야해요
페너
틀어주면
남겼습니다
모아에서
카토
재밌네
먹었다구요
닥
ㄱㅊㄱㅊ
같긴한데
원점
싶긴
제비
료워
따리
들어서면
본건데
점심때
갈게
날땐
없겠다
산화
않더라고요
콜
벗고
기침
지역
부드럽게
12
이었나
이었나
08
즐기면서
쩡
팔방
아니시라면
나를
살고있어서
대형견
정돜
스트로
도도
일일이
찍어주시는
도도
일일이
찍어주시는
츠
화분
보일듯
끼어
어둑한
출입문
목격
그려진
어리둥절했음
살펴
성공한
알았다더니
완벽할듯
짜고
나왔던거
치웠으면
고함
올라
위아래
어떤게
연하
인식
신선한것
바라지는
냄
백도
비아
수유역
등급
석양
있는것만
어요
대량
클레
몰려와서
시끄러워
터눈
나라
해유
모르겟구
림
쥐
생겼고
공중목욕탕
않겠음
호스
달라붙지
없었던
빌려주셔서
맞았어요
블치케
물어봐
혐오
만들어주셨어요
타운
본데요
넘은
문화유산
부속
파크
당하고
뎀
올랐어요
까지
뒀으면
마련
무려
루이보스
채소
버령
화차
뽑는거
곤욕
은애
곡
딱딱하지
해먹
용인
하신거죠
무는
넓게
보고있을
웃으면서
바사삭
폭탄
이였던
넣지를
행주
환경호르몬
않았구
눌러주신
머뭇
좋아했던
키치
튀는
프롤라
목도
크하는
귀엽긴
상봉
dense
공중
다임
유명하지만
특수
왕성
듣는
발라드
나왔으면
가슴
반겨주는
맛없을
스토어
다다다
그러실
천지
그러다
낼
기억남네요
양반
오더라
끝내기
오더라고
부족해서
평준화
오심
먹을게요
가급적
조름한
간질간질
졸고
스르르
평화
고요한
미술관
나위
맑은
3일
낫다
마사지
6월
끝내
도르랑
미학
버리네요
없네여
쥐
선풍기
건강하게
돌아오세

### 전처리 완료 후 다시 df에 합치기

In [29]:
# 토큰 단위로 쪼개져있던 리뷰를 문장으로 다시 합쳐주기
review_list = []

for review in review_tokenized:
    sentence = " ".join(review)
    review_list.append(sentence)

review_list[:5]

['커피 원두 맛남',
 '에그타르트 클래식 에그타르트 크림 치즈 5 시나몬 향 나고 빵 부분 계란 맛 좀 적게 느껴진다',
 '나 는 디저트 너무 달았다 ㅜ',
 '플랫 화이트 그럭저럭',
 'nan']

In [30]:
# review_series: review_tokenized의 자료구조 변경 (list -> series)
review_series = pd.Series(review_list)
review_series.head()

0                                             커피 원두 맛남
1    에그타르트 클래식 에그타르트 크림 치즈 5 시나몬 향 나고 빵 부분 계란 맛 좀 적...
2                                     나 는 디저트 너무 달았다 ㅜ
3                                          플랫 화이트 그럭저럭
4                                                  nan
dtype: object

In [31]:
# df의 reviews 컬럼을 review_series로 변경
df["reviews"] = review_series
df.head()

,userIndex,itemIndex,rating,reviews
0,100i22n,고로커피로스터스 봉천동,4.0,커피 원두 맛남
1,100i22n,나따오비까 대치점 대치동,3.0,에그타르트 클래식 에그타르트 크림 치즈 5 시나몬 향 나고 빵 부분 계란 맛 좀 적...
2,100i22n,뉴욕택시디저트 봉천동,2.0,나 는 디저트 너무 달았다 ㅜ
3,100i22n,델트 구의동,3.0,플랫 화이트 그럭저럭
4,100i22n,카페산다 봉천동,3.0,nan


### 각 컬럼별로 자료형 맞춰주기

In [32]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7041 entries, 0 to 7040
Data columns (total 4 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   userIndex  7041 non-null   object 
 1   itemIndex  7041 non-null   object 
 2   rating     7041 non-null   float64
 3   reviews    7041 non-null   object 
dtypes: float64(1), object(3)
memory usage: 220.2+ KB


#### rating

In [33]:
# rating: float64 -> int64
df = df.astype({"rating":int})
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7041 entries, 0 to 7040
Data columns (total 4 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   userIndex  7041 non-null   object
 1   itemIndex  7041 non-null   object
 2   rating     7041 non-null   int64 
 3   reviews    7041 non-null   object
dtypes: int64(1), object(3)
memory usage: 220.2+ KB


In [34]:
pd.Series(df["userIndex"].unique())

0      100i22n
1      100q04c
2      101iuk0
3      1034meq
4      103hblo
        ...   
799     vpqam5
800     vpqjrc
801     vqavpg
802     vqsnrv
803     vur3vt
Length: 804, dtype: object

In [35]:
# df_userIndex: 기존 userIndex를 숫자 인덱스로 매칭
df_userIndex = pd.DataFrame(df["userIndex"].unique())
df_userIndex["NumberIndex"] = df_userIndex.index
df_userIndex.columns = ["StringIndex", "NumberIndex"]
df_userIndex.head()

,StringIndex,NumberIndex
0,100i22n,0
1,100q04c,1
2,101iuk0,2
3,1034meq,3
4,103hblo,4


In [36]:
# df_itemIndex: 기존 itemIndex를 숫자 인덱스로 매칭
df_itemIndex = pd.DataFrame(df["itemIndex"].unique())
df_itemIndex["NumberIndex"] = df_itemIndex.index
df_itemIndex.columns = ["StringIndex", "NumberIndex"]
df_itemIndex.head()

,StringIndex,NumberIndex
0,고로커피로스터스 봉천동,0
1,나따오비까 대치점 대치동,1
2,뉴욕택시디저트 봉천동,2
3,델트 구의동,3
4,카페산다 봉천동,4


In [37]:
# df_userIndex, df_itemIndex 저장
df_userIndex.to_csv('/content/drive/MyDrive/DSL/[프로젝트] 모델링/A3NCF/data preprocessing/rawdata_for_A3NCF/review_five_user.csv', index=False)
df_itemIndex.to_csv('/content/drive/MyDrive/DSL/[프로젝트] 모델링/A3NCF/data preprocessing/rawdata_for_A3NCF/review_five_item.csv', index=False)

In [38]:
# 저장한 csv 파일 확인
_user = pd.read_csv('/content/drive/MyDrive/DSL/[프로젝트] 모델링/A3NCF/data preprocessing/rawdata_for_A3NCF/review_five_user.csv')
_item = pd.read_csv('/content/drive/MyDrive/DSL/[프로젝트] 모델링/A3NCF/data preprocessing/rawdata_for_A3NCF/review_five_item.csv')
_item[:5]

,StringIndex,NumberIndex
0,고로커피로스터스 봉천동,0
1,나따오비까 대치점 대치동,1
2,뉴욕택시디저트 봉천동,2
3,델트 구의동,3
4,카페산다 봉천동,4


In [39]:
# dict_userIndex, dict_itemIndex: df_userIndex, df_itemIndex의 dictionary 버전
dict_userIndex = df_userIndex.set_index("StringIndex").T.to_dict("records")
dict_userIndex = dict_userIndex[0]
dict_itemIndex = df_itemIndex.set_index("StringIndex").T.to_dict("records")
dict_itemIndex = dict_itemIndex[0]
type(dict_userIndex), type(dict_itemIndex)

(dict, dict)

In [40]:
col_user = list(df["userIndex"])
col_item = list(df["itemIndex"])

In [41]:
for i, user in enumerate(col_user):
    for key in dict_userIndex.keys():
        if user == key:
            col_user[i] = dict_userIndex[key]

In [42]:
for i, item in enumerate(col_item):
    for key in dict_itemIndex.keys():
        if item == key:
            col_item[i] = dict_itemIndex[key]

#### userIndex, itemIndex

In [43]:
# userIndex, itemIndex: object -> int
df["userIndex"] = col_user
df["itemIndex"] = col_item

In [44]:
df.head()

,userIndex,itemIndex,rating,reviews
0,0,0,4,커피 원두 맛남
1,0,1,3,에그타르트 클래식 에그타르트 크림 치즈 5 시나몬 향 나고 빵 부분 계란 맛 좀 적...
2,0,2,2,나 는 디저트 너무 달았다 ㅜ
3,0,3,3,플랫 화이트 그럭저럭
4,0,4,3,nan


In [45]:
df.columns = [0, 1, 2, 3]
df

,0,1,2,3
0,0,0,4,커피 원두 맛남
1,0,1,3,에그타르트 클래식 에그타르트 크림 치즈 5 시나몬 향 나고 빵 부분 계란 맛 좀 적...
2,0,2,2,나 는 디저트 너무 달았다 ㅜ
3,0,3,3,플랫 화이트 그럭저럭
4,0,4,3,nan
...,...,...,...,...
7036,803,153,1,콘센트 못 다 막아놓음 불친절하고 자꾸 아니 게 맞는 듯
7037,803,148,5,친절하시고 맛있습니다 딸기 케이크 존맛 탱
7038,803,478,4,그냥 보통 사람 많고 해요 근데 언제 빵빵 와이파이 카페 됐나요 충전 코드 것 아니...
7039,803,155,5,직원 친절하세요 음료 맛있어요


In [46]:
# 최종 확인
# A3NCF_dataset_test에서 확인한 df_tm.info()와 동일해야함
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7041 entries, 0 to 7040
Data columns (total 4 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   0       7041 non-null   int64 
 1   1       7041 non-null   int64 
 2   2       7041 non-null   int64 
 3   3       7041 non-null   object
dtypes: int64(3), object(1)
memory usage: 220.2+ KB


## 전처리 완료된 데이터 내보내기

In [47]:
# 모든 전처리가 완료된 df 내보내기 !
#df.to_csv('/content/drive/MyDrive/DSL/[프로젝트] 모델링/A3NCF/topic model/data/A3NCF_topicmodel_input_df_5.csv', 
#          index=False)           # row명 (index) 저장하지 않기

In [48]:
# 내보낸 df 확인
#df = pd.read_csv("/content/drive/MyDrive/DSL/[프로젝트] 모델링/A3NCF/topic model/data/A3NCF_topicmodel_input_df_5.csv")
#df

In [49]:
# hoxy..?
df.to_csv('/content/drive/MyDrive/DSL/[프로젝트] 모델링/A3NCF/topic model/data/A3NCF_topicmodel_input_df_5.dat', 
          index=False)           # row명 (index) 저장하지 않기

In [50]:
# hehe
df = pd.read_csv("/content/drive/MyDrive/DSL/[프로젝트] 모델링/A3NCF/topic model/data/A3NCF_topicmodel_input_df_5.dat")
df

,0,1,2,3
0,0,0,4,커피 원두 맛남
1,0,1,3,에그타르트 클래식 에그타르트 크림 치즈 5 시나몬 향 나고 빵 부분 계란 맛 좀 적...
2,0,2,2,나 는 디저트 너무 달았다 ㅜ
3,0,3,3,플랫 화이트 그럭저럭
4,0,4,3,NaN
...,...,...,...,...
7036,803,153,1,콘센트 못 다 막아놓음 불친절하고 자꾸 아니 게 맞는 듯
7037,803,148,5,친절하시고 맛있습니다 딸기 케이크 존맛 탱
7038,803,478,4,그냥 보통 사람 많고 해요 근데 언제 빵빵 와이파이 카페 됐나요 충전 코드 것 아니...
7039,803,155,5,직원 친절하세요 음료 맛있어요
